# Logic Grid Puzzle Solver - Test 100 Puzzles
This notebook solves 3x3 logic grid puzzles using a permutation-based solver with constraint filtering.

## Features
- Enumerates all name/color/pet permutations for 3x3 puzzles (exact search, deterministic)
- Forward checking: filters candidates by regex-parsed clue constraints in clue order
- MRV-style bitmask search to select a consistent candidate with minimal decision steps
- Regex clue parsing into positional, ownership, adjacency, and exclusion constraints
- Batch solve of the dataset with JSON export and sample/diagnostic prints

In [26]:
# Imports and inline solver (no external imports of solver functions)
import pandas as pd
import json
import re
from dataclasses import dataclass
from functools import lru_cache
from itertools import permutations
from typing import Dict, List, Optional, Sequence, Tuple, Callable, Any

# --- Parser helpers (from parser.py) ---
KNOWN_NAMES = {
    "Alice", "Bob", "Carol", "David", "Eve", "Frank",
    "Grace", "Heidi", "Ivan", "Judy", "Mallory", "Niaj",
}


def norm_text(s: str) -> str:
    s = s.strip()
    s = re.sub(r"\s+", " ", s)
    if s.endswith('.'):
        s = s[:-1]
    return s


def parse_list_line(line: str, key: str) -> Optional[List[str]]:
    m = re.search(rf"^{re.escape(key)}:\s*(.+?)\.?$", line.strip(), flags=re.IGNORECASE)
    if not m:
        return None
    return [x.strip().lower() for x in m.group(1).split(',') if x.strip()]


def extract_clues(puzzle_text: str) -> List[str]:
    in_clues = False
    clues: List[str] = []
    for raw_line in puzzle_text.splitlines():
        line = raw_line.strip()
        if not line:
            continue
        if line.lower().startswith('clues:'):
            in_clues = True
            continue
        if not in_clues:
            continue
        m = re.match(r"^\d+\.\s*(.+)$", line)
        if not m:
            continue
        clues.append(norm_text(m.group(1)))
    return clues


def extract_names_from_clues(clues: Sequence[str]) -> List[str]:
    names_set = set()
    for clue in clues:
        for name in KNOWN_NAMES:
            if re.search(rf"\b{re.escape(name)}\b", clue, flags=re.IGNORECASE):
                names_set.add(name)
    if not names_set:
        for clue in clues:
            m = re.match(r"^(?P<name>[A-Z][a-z]+)\b", clue)
            if m:
                n = m.group('name')
                if n not in ("The", "House"):
                    names_set.add(n)
    return sorted(names_set)


# --- Candidate model (from solver.py) ---
@dataclass(frozen=True)
class Candidate:
    size: int
    names_by_house: Tuple[str, ...]
    colors_by_house: Tuple[str, ...]
    pets_by_house: Tuple[str, ...]

    def to_grid_json(self) -> Dict[str, object]:
        return {
            "header": ["House", "Name", "Color", "Pet"],
            "rows": [
                [str(i), self.names_by_house[i - 1], self.colors_by_house[i - 1], self.pets_by_house[i - 1]]
                for i in range(1, self.size + 1)
            ],
        }


Constraint = Callable[[Candidate], bool]


def build_constraints(clues: Sequence[str], colors: Sequence[str], pets: Sequence[str]) -> Tuple[List[Constraint], List[str]]:
    colors_set = set(colors)
    pets_set = set(pets)

    constraints: List[Constraint] = []
    unknown: List[str] = []

    def add(fn: Constraint) -> None:
        constraints.append(fn)

    for raw in clues:
        c = raw

        m = re.match(r"^(?P<name>[A-Z][a-z]+) lives in house (?P<house>\d+)$", c)
        if m:
            name = m.group('name')
            house = int(m.group('house'))

            def _fn(candidate: Candidate, name=name, house=house) -> bool:
                return candidate.names_by_house[house - 1] == name

            add(_fn)
            continue

        m = re.match(r"^(?P<name>[A-Z][a-z]+) lives in the (?P<color>[a-z]+) house$", c)
        if m:
            name = m.group('name')
            color = m.group('color').lower()
            if color not in colors_set:
                unknown.append(raw)
                continue

            def _fn(candidate: Candidate, name=name, color=color) -> bool:
                return candidate.names_by_house[candidate.colors_by_house.index(color)] == name

            add(_fn)
            continue

        m = re.match(r"^House (?P<house>\d+) is painted (?P<color>[a-z]+)$", c)
        if m:
            house = int(m.group('house'))
            color = m.group('color').lower()
            if color not in colors_set:
                unknown.append(raw)
                continue

            def _fn(candidate: Candidate, house=house, color=color) -> bool:
                return candidate.colors_by_house[house - 1] == color

            add(_fn)
            continue

        m = re.match(r"^The (?P<left>[a-z]+) house is immediately to the left of the (?P<right>[a-z]+) house$", c)
        if m:
            left = m.group('left').lower()
            right = m.group('right').lower()
            if left not in colors_set or right not in colors_set:
                unknown.append(raw)
                continue

            def _fn(candidate: Candidate, left=left, right=right) -> bool:
                return candidate.colors_by_house.index(left) + 1 == candidate.colors_by_house.index(right)

            add(_fn)
            continue

        m = re.match(r"^The (?P<right>[a-z]+) house is immediately to the right of the (?P<left>[a-z]+) house$", c)
        if m:
            right = m.group('right').lower()
            left = m.group('left').lower()
            if left not in colors_set or right not in colors_set:
                unknown.append(raw)
                continue

            def _fn(candidate: Candidate, left=left, right=right) -> bool:
                return candidate.colors_by_house.index(right) - 1 == candidate.colors_by_house.index(left)

            add(_fn)
            continue

        m = re.match(r"^The (?P<color>[a-z]+) house contains the (?P<pet>[a-z]+)$", c)
        if m:
            color = m.group('color').lower()
            pet = m.group('pet').lower()
            if color not in colors_set or pet not in pets_set:
                unknown.append(raw)
                continue

            def _fn(candidate: Candidate, color=color, pet=pet) -> bool:
                idx = candidate.colors_by_house.index(color)
                return candidate.pets_by_house[idx] == pet

            add(_fn)
            continue

        m = re.match(r"^(?P<name>[A-Z][a-z]+) lives in the (?P<color>[a-z]+) house$", c)
        if m:
            name = m.group('name')
            color = m.group('color').lower()
            if color not in colors_set:
                unknown.append(raw)
                continue

            def _fn(candidate: Candidate, name=name, color=color) -> bool:
                return candidate.names_by_house[candidate.colors_by_house.index(color)] == name

            add(_fn)
            continue

        m = re.match(r"^The person in house (?P<house>\d+) owns the (?P<pet>[a-z]+)$", c)
        if m:
            house = int(m.group('house'))
            pet = m.group('pet').lower()
            if pet not in pets_set:
                unknown.append(raw)
                continue

            def _fn(candidate: Candidate, house=house, pet=pet) -> bool:
                return candidate.pets_by_house[house - 1] == pet

            add(_fn)
            continue

        m = re.match(r"^(?P<name>[A-Z][a-z]+) owns the (?P<pet>[a-z]+)$", c)
        if m:
            name = m.group('name')
            pet = m.group('pet').lower()
            if pet not in pets_set:
                unknown.append(raw)
                continue

            def _fn(candidate: Candidate, name=name, pet=pet) -> bool:
                idx = candidate.names_by_house.index(name)
                return candidate.pets_by_house[idx] == pet

            add(_fn)
            continue

        m = re.match(r"^(?P<name>[A-Z][a-z]+) does not live in the (?P<color>[a-z]+) house$", c)
        if m:
            name = m.group('name')
            color = m.group('color').lower()
            if color not in colors_set:
                unknown.append(raw)
                continue

            def _fn(candidate: Candidate, name=name, color=color) -> bool:
                return candidate.names_by_house.index(name) != candidate.colors_by_house.index(color)

            add(_fn)
            continue

        m = re.match(r"^(?P<name>[A-Z][a-z]+) is not in the (?P<color>[a-z]+) house$", c)
        if m:
            name = m.group('name')
            color = m.group('color').lower()
            if color not in colors_set:
                unknown.append(raw)
                continue

            def _fn(candidate: Candidate, name=name, color=color) -> bool:
                return candidate.names_by_house.index(name) != candidate.colors_by_house.index(color)

            add(_fn)
            continue

        m = re.match(r"^The (?P<a>[a-z]+) house is next to the (?P<b>[a-z]+) house$", c)
        if m:
            a = m.group('a').lower()
            b = m.group('b').lower()
            if a not in colors_set or b not in colors_set:
                unknown.append(raw)
                continue

            def _fn(candidate: Candidate, a=a, b=b) -> bool:
                return abs(candidate.colors_by_house.index(a) - candidate.colors_by_house.index(b)) == 1

            add(_fn)
            continue

        unknown.append(raw)

    return constraints, unknown


# --- Candidate generation and search ---
def generate_candidates(size: int, names: Sequence[str], colors: Sequence[str], pets: Sequence[str]) -> List[Candidate]:
    return [
        Candidate(size=size, names_by_house=tuple(nm), colors_by_house=tuple(cl), pets_by_house=tuple(pt))
        for nm in permutations(names, size)
        for cl in permutations(colors, size)
        for pt in permutations(pets, size)
    ]


def _lsb_index(x: int) -> int:
    return (x & -x).bit_length() - 1


def solve_min_steps(candidates: List[Candidate]) -> Tuple[Optional[Candidate], Optional[int]]:
    if not candidates:
        return None, None

    n = len(candidates)
    full_mask = (1 << n) - 1

    kinds = ("Name", "Color", "Pet")
    size = candidates[0].size

    values_by_kind: Dict[str, List[str]] = {
        "Name": sorted({v for c in candidates for v in c.names_by_house}),
        "Color": sorted({v for c in candidates for v in c.colors_by_house}),
        "Pet": sorted({v for c in candidates for v in c.pets_by_house}),
    }

    var_value_mask: Dict[Tuple[str, int, str], int] = {}
    for kind in kinds:
        for house in range(1, size + 1):
            for value in values_by_kind[kind]:
                var_value_mask[(kind, house, value)] = 0

    for idx, cand in enumerate(candidates):
        bit = 1 << idx
        for house in range(1, size + 1):
            var_value_mask[("Name", house, cand.names_by_house[house - 1])] |= bit
            var_value_mask[("Color", house, cand.colors_by_house[house - 1])] |= bit
            var_value_mask[("Pet", house, cand.pets_by_house[house - 1])] |= bit

    vars_all: List[Tuple[str, int]] = [(k, h) for k in kinds for h in range(1, size + 1)]

    def domain_values(mask: int, var: Tuple[str, int]) -> List[str]:
        kind, house = var
        vals: List[str] = []
        for value in values_by_kind[kind]:
            if mask & var_value_mask[(kind, house, value)]:
                vals.append(value)
        return vals

    @lru_cache(maxsize=None)
    def search(mask: int) -> Tuple[int, int]:
        if mask == 0:
            return 10**9, -1
        if mask & (mask - 1) == 0:
            return 0, _lsb_index(mask)

        best_var: Optional[Tuple[str, int]] = None
        best_dom: Optional[List[str]] = None
        best_size = 10**9
        for var in vars_all:
            dom = domain_values(mask, var)
            if len(dom) <= 1:
                continue
            if len(dom) < best_size:
                best_size = len(dom)
                best_var = var
                best_dom = dom
                if best_size == 2:
                    break

        if best_var is None or best_dom is None:
            return 10**9, -1

        kind, house = best_var
        best_steps = 10**9
        best_idx = -1
        for value in best_dom:
            child_mask = mask & var_value_mask[(kind, house, value)]
            steps_child, idx_child = search(child_mask)
            if idx_child == -1:
                continue
            total = 1 + steps_child
            if total < best_steps:
                best_steps = total
                best_idx = idx_child

        return best_steps, best_idx

    steps, idx = search(full_mask)
    if idx == -1 or steps >= 10**9:
        return None, None
    return candidates[idx], steps


# --- Public solve API ---
def solve_puzzle_text(puzzle_text: str, size_raw: str) -> Dict[str, object]:
    m = re.match(r"^(\d+)\*(\d+)$", size_raw.strip())
    if not m or m.group(1) != m.group(2):
        return {"status": "error", "error": f"Unsupported size: {size_raw}"}

    size = int(m.group(1))
    if size != 3:
        return {"status": "error", "error": f"Only 3*3 supported right now; got {size_raw}"}

    colors: Optional[List[str]] = None
    pets: Optional[List[str]] = None
    for line in puzzle_text.splitlines():
        if colors is None:
            colors = parse_list_line(line, "Colors")
        if pets is None:
            pets = parse_list_line(line, "Pets")

    if not colors or not pets:
        return {"status": "error", "error": "Missing Colors/Pets lists"}

    clues = extract_clues(puzzle_text)
    names = extract_names_from_clues(clues)
    issues: List[str] = []

    if len(names) < size:
        for i in range(size - len(names)):
            names.append(f"Unknown{i + 1}")
        issues.append("incomplete_people")
    elif len(names) > size:
        issues.append("too_many_people")
        names = names[:size]

    constraints, unknown_clues = build_constraints(clues, colors, pets)
    if unknown_clues:
        issues.append(f"unparsed_clues:{len(unknown_clues)}")

    all_candidates = generate_candidates(size=size, names=names, colors=colors, pets=pets)

    forward_check_steps = 0
    feasible = all_candidates
    for con in constraints:
        before = len(feasible)
        feasible = [c for c in feasible if con(c)]
        after = len(feasible)
        if after < before:
            forward_check_steps += 1

    solution, decision_steps = solve_min_steps(feasible)
    if solution is None:
        return {"status": "unsatisfiable", "issues": issues, "grid_solution": None, "steps": None}

    steps = forward_check_steps + (decision_steps if decision_steps is not None else 0)

    return {
        "status": "ok",
        "issues": issues,
        "grid_solution": solution.to_grid_json(),
        "steps": steps,
        "forward_check_steps": forward_check_steps,
        "decision_steps": decision_steps,
        "num_solutions": len(feasible),
    }


In [27]:
def solve_puzzle(puzzle_data: Dict[str, Any]) -> Dict[str, Any]:
    """Solve a single logic grid puzzle using the validated inline solver logic."""
    try:
        puzzle_text = puzzle_data.get('puzzle', '')
        size_raw = str(puzzle_data.get('size', '')).strip()

        result = solve_puzzle_text(puzzle_text, size_raw)
        if result.get('status') != 'ok':
            return {'error': result.get('error', 'No solution found')}

        grid_solution = result.get('grid_solution', {})
        headers = grid_solution.get('header', [])
        rows_in = grid_solution.get('rows', [])

        rows_out = []
        for row in rows_in:
            house, name, color, pet = row
            if isinstance(name, str) and name.lower().startswith('unknown'):
                name_fmt = ''
            else:
                name_fmt = name.capitalize() if isinstance(name, str) else name
            rows_out.append([str(house), name_fmt, color, pet])

        return {
            'headers': headers,
            'grid': rows_out,
            'steps': result.get('steps'),
            'forward_check_steps': result.get('forward_check_steps'),
            'decision_steps': result.get('decision_steps'),
            'num_solutions': result.get('num_solutions')
        }

    except Exception as e:
        return {'error': f'Error solving puzzle: {str(e)}'}


## Main Execution
Load Test_100_Puzzles.csv and solve all puzzles.

In [28]:
# Load Test_100_Puzzles.parquet
df = pd.read_parquet('Test_100_Puzzles.parquet')

print(f"Loaded {len(df)} puzzles from Test_100_Puzzles.parquet")
print(f"Columns: {list(df.columns)}")
print(f"\nFirst puzzle:")
print(df.iloc[0]['puzzle'][:500] + "...")

Loaded 100 puzzles from Test_100_Puzzles.parquet
Columns: ['id', 'size', 'puzzle', 'created_at']

First puzzle:
Three friends live in three houses in a row, numbered 1 to 3. Each house is painted a different color and each friend owns a different pet.

Colors: orange, blue, green.
Pets: cat, turtle, dog.

Clues:
1. Mallory lives in the blue house.
2. Alice lives in house 3.
3. The orange house contains the turtle.
4. House 1 is painted orange.
5. Bob does not live in the blue house.
6. Mallory does not live in the orange house....


In [29]:
# Solve all puzzles with hybrid approach (no name variables, names inferred post-solve)
results = {}

for idx, row in df.iterrows():
    puzzle_id = row['id']
    puzzle_data = {
        'id': puzzle_id,
        'size': row['size'],
        'puzzle': row['puzzle']
    }
    
    result = solve_puzzle(puzzle_data)
    results[puzzle_id] = result

solved_count = sum(1 for r in results.values() if r is not None and 'headers' in r)
total_puzzles = len(results)
success_rate = (solved_count / total_puzzles * 100) if total_puzzles > 0 else 0

print(f"\nSolved {solved_count}/{total_puzzles} puzzles ({success_rate:.1f}%)")


Solved 100/100 puzzles (100.0%)


In [30]:
# Save results to results.json
output_file = 'results.json'

with open(output_file, 'w') as f:
    json.dump(results, f, indent=2)

print(f"\nResults saved to {output_file}")
print(f"File contains {len(results)} puzzle solutions")


Results saved to results.json
File contains 100 puzzle solutions


In [31]:
# Display a sample solution (first solved puzzle)
for puzzle_id, solution in results.items():
    if solution is not None and 'headers' in solution:
        print(f"\nSample Solution for {puzzle_id}:")
        print("=" * 80)
        
        headers = solution['headers']
        rows = solution['grid']
        
        # Calculate column widths
        col_widths = [len(str(h)) for h in headers]
        for row in rows:
            for i, cell in enumerate(row):
                col_widths[i] = max(col_widths[i], len(str(cell)))
        
        # Print table
        header_str = " | ".join(str(h).ljust(col_widths[i]) for i, h in enumerate(headers))
        print(f"| {header_str} |")
        print("|" + "-+-".join("-" * w for w in col_widths) + "|")
        
        for row in rows:
            row_str = " | ".join(str(cell).ljust(col_widths[i]) for i, cell in enumerate(row))
            print(f"| {row_str} |")
        
        print("=" * 80)
        break


Sample Solution for test-3x3-001:
| House | Name    | Color  | Pet    |
|------+---------+--------+-------|
| 1     | Bob     | orange | turtle |
| 2     | Mallory | blue   | cat    |
| 3     | Alice   | green  | dog    |


In [32]:
# TEST: Check if fixes work on test-3x3-002 (previously missing name column)
test_id = 'test-3x3-002'
test_row = df[df['id'] == test_id]

if not test_row.empty:
    test_data = {
        'id': test_row.iloc[0]['id'],
        'size': test_row.iloc[0]['size'],
        'puzzle': test_row.iloc[0]['puzzle']
    }
    
    print(f"Testing {test_id}...")
    print("=" * 80)
    print("Puzzle:")
    print(test_data['puzzle'][:400])
    print("\n" + "-" * 80)
    
    test_result = solve_puzzle(test_data)
    
    if 'error' not in test_result:
        print("\n✓ SOLVED!")
        print(f"\nHeaders: {test_result['headers']}")
        for row in test_result['grid']:
            print(f"  {row}")
        
        # Check if name column is present
        if 'Name' in test_result['headers']:
            print("\n✓✓✓ SUCCESS: Name column is now present! ✓✓✓")
        else:
            print("\n⚠ WARNING: Name column is still missing")
    else:
        print(f"\n✗ FAILED TO SOLVE: {test_result['error']}")
else:
    print(f"Puzzle {test_id} not found in dataset")

Testing test-3x3-002...
Puzzle:
Three friends live in three houses in a row, numbered 1 to 3. Each house is painted a different color and each friend owns a different pet.

Colors: red, blue, orange.
Pets: fish, bird, dog.

Clues:
1. The person in house 3 owns the bird.
2. The red house is immediately to the left of the orange house.
3. The blue house contains the dog.
4. The person in house 2 owns the fish.
5. Niaj does not liv

--------------------------------------------------------------------------------

✓ SOLVED!

Headers: ['House', 'Name', 'Color', 'Pet']
  ['1', 'Bob', 'blue', 'dog']
  ['2', 'Niaj', 'red', 'fish']
  ['3', '', 'orange', 'bird']

✓✓✓ SUCCESS: Name column is now present! ✓✓✓


In [33]:
# DIAGNOSTIC: Solve a single puzzle using the reference solver logic

test_id = 'test-3x3-001'
test_row = df[df['id'] == test_id]

if test_row.empty:
    print(f"Puzzle {test_id} not found in dataset")
else:
    puzzle_data = {
        'id': test_row.iloc[0]['id'],
        'size': test_row.iloc[0]['size'],
        'puzzle': test_row.iloc[0]['puzzle']
    }

    print(f"=== DIAGNOSTIC FOR {test_id} ===\n")
    print("PUZZLE TEXT:")
    print(puzzle_data['puzzle'])
    print("\n" + "="*80 + "\n")

    # Use the unified solve_puzzle (which delegates to solver.py logic)
    diag_result = solve_puzzle(puzzle_data)

    if 'error' in diag_result:
        print(f"✗ NO SOLUTION FOUND: {diag_result['error']}")
    else:
        print("✓ SOLVED!")
        print(f"Steps: {diag_result.get('steps')} (forward: {diag_result.get('forward_check_steps')}, decision: {diag_result.get('decision_steps')})")
        print(f"Num solutions considered: {diag_result.get('num_solutions')}")
        print("\nSOLUTION:")
        for row in diag_result['grid']:
            print(f"  {row}")


=== DIAGNOSTIC FOR test-3x3-001 ===

PUZZLE TEXT:
Three friends live in three houses in a row, numbered 1 to 3. Each house is painted a different color and each friend owns a different pet.

Colors: orange, blue, green.
Pets: cat, turtle, dog.

Clues:
1. Mallory lives in the blue house.
2. Alice lives in house 3.
3. The orange house contains the turtle.
4. House 1 is painted orange.
5. Bob does not live in the blue house.
6. Mallory does not live in the orange house.


✓ SOLVED!
Steps: 5 (forward: 4, decision: 1)
Num solutions considered: 2

SOLUTION:
  ['1', 'Bob', 'orange', 'turtle']
  ['2', 'Mallory', 'blue', 'cat']
  ['3', 'Alice', 'green', 'dog']
